<a href="https://colab.research.google.com/github/plotsaf8/first/blob/main/preprocessing_%D0%91%D0%B5%D1%80%D0%B5%D0%B7%D0%BE%D0%B2%D1%81%D0%BA%D0%B0%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install numpy
!pip install pymorphy2

import numpy as np
import pandas as pd
import re
import nltk
import string

##Чтение датасета с текстом
df = pd.read_csv("/content/kinopoisk-top250.csv", encoding='UTF8')
df.head(10)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 98.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=06c6c29be6cd5877c683234fdcbdb78437a5b8bc718a3df533e8d9c767b39d7b
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


,rating,movie,year,country,rating_ball,overview,director,screenwriter,actors,url_logo
0,0,Побег из Шоушенка,1994,США,9.111,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Тим Роббинс; Морган Фриман; Боб Гантон; Уильям...,'https://st.kp.yandex.net/images/film_iphone/i...
1,1,Зеленая миля,1999,США,9.062,Пол Эджкомб — начальник блока смертников в тюр...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Том Хэнкс; Дэвид Морс; Бонни Хант; Майкл Кларк...,'https://st.kp.yandex.net/images/film_iphone/i...
2,2,Форрест Гамп,1994,США,8.913,От лица главного героя Форреста Гампа; слабоум...,Роберт Земекис,Эрик Рот; Уинстон Грум,Том Хэнкс; Робин Райт; Салли Филд; Гэри Синиз;...,'https://st.kp.yandex.net/images/film_iphone/i...
3,3,Список Шиндлера,1993,США,8.817,Фильм рассказывает реальную историю загадочног...,Стивен Спилберг,Стивен Зеллиан; Томас Кенилли,Лиам Нисон; Бен Кингсли; Рэйф Файнс; Кэролайн ...,'https://st.kp.yandex.net/images/film_iphone/i...
4,4,1+1,2011,Франция,8.807,Пострадав в результате несчастного случая; бог...,Оливье Накаш; Эрик Толедано,Оливье Накаш; Филипп Поццо ди Борго; Эрик Т...,Франсуа Клюзе; Омар Си; Анн Ле Ни; Одри Флеро;...,'https://st.kp.yandex.net/images/film_iphone/i...
5,5,Начало,2010,США,8.662,Кобб — талантливый вор; лучший из лучших в опа...,Кристофер Нолан,Кристофер Нолан,Леонардо ДиКаприо; Джозеф Гордон-Левитт; Эллен...,'https://st.kp.yandex.net/images/film_iphone/i...
6,6,Леон,1994,Франция,8.681,Профессиональный убийца Леон; не знающий пощад...,Люк Бессон,Люк Бессон,Жан Рено; Гари Олдман; Натали Портман; Дэнни А...,'https://st.kp.yandex.net/images/film_iphone/i...
7,7,Король Лев,1994,США,8.772,У величественного Короля-Льва Муфасы рождается...,Роджер Аллерс; Роб Минкофф,Ирен Меччи; Джонатан Робертс; Линда Вулверт...,Мэттью Бродерик; Джереми Айронс; Нэйтан Лейн; ...,'https://st.kp.yandex.net/images/film_iphone/i...
8,8,Бойцовский клуб,1999,США,8.645,Сотрудник страховой компании страдает хроничес...,Дэвид Финчер,Джим Улс; Чак Паланик,Эдвард Нортон; Брэд Питт; Хелена Бонем Картер;...,'https://st.kp.yandex.net/images/film_iphone/i...
9,9,Иван Васильевич меняет профессию,1973,СССР,8.782,Инженер-изобретатель Тимофеев сконструировал м...,Леонид Гайдай,Владлен Бахнов; Леонид Гайдай; Михаил Булгаков,Александр Демьяненко; Юрий Яковлев; Леонид Кур...,'https://st.kp.yandex.net/images/film_iphone/i...


In [24]:
#Нижний регистр. Например, Товар --> товар, УРА --> ура
df['text_lower']  = df['overview'].str.lower()
df['text_lower'].head()
df.head()

,rating,movie,year,country,rating_ball,overview,director,screenwriter,actors,url_logo,text_lower
0,0,Побег из Шоушенка,1994,США,9.111,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Тим Роббинс; Морган Фриман; Боб Гантон; Уильям...,'https://st.kp.yandex.net/images/film_iphone/i...,бухгалтер энди дюфрейн обвинён в убийстве собс...
1,1,Зеленая миля,1999,США,9.062,Пол Эджкомб — начальник блока смертников в тюр...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Том Хэнкс; Дэвид Морс; Бонни Хант; Майкл Кларк...,'https://st.kp.yandex.net/images/film_iphone/i...,пол эджкомб — начальник блока смертников в тюр...
2,2,Форрест Гамп,1994,США,8.913,От лица главного героя Форреста Гампа; слабоум...,Роберт Земекис,Эрик Рот; Уинстон Грум,Том Хэнкс; Робин Райт; Салли Филд; Гэри Синиз;...,'https://st.kp.yandex.net/images/film_iphone/i...,от лица главного героя форреста гампа; слабоум...
3,3,Список Шиндлера,1993,США,8.817,Фильм рассказывает реальную историю загадочног...,Стивен Спилберг,Стивен Зеллиан; Томас Кенилли,Лиам Нисон; Бен Кингсли; Рэйф Файнс; Кэролайн ...,'https://st.kp.yandex.net/images/film_iphone/i...,фильм рассказывает реальную историю загадочног...
4,4,1+1,2011,Франция,8.807,Пострадав в результате несчастного случая; бог...,Оливье Накаш; Эрик Толедано,Оливье Накаш; Филипп Поццо ди Борго; Эрик Т...,Франсуа Клюзе; Омар Си; Анн Ле Ни; Одри Флеро;...,'https://st.kp.yandex.net/images/film_iphone/i...,пострадав в результате несчастного случая; бог...


In [25]:
#Удаление пунктуации
df['text_punct'] = df['text_lower'].str.replace('[^\w\s]','')
df['text_punct'].head()

,text_punct
0,бухгалтер энди дюфрейн обвинён в убийстве собс...
1,пол эджкомб — начальник блока смертников в тюр...
2,от лица главного героя форреста гампа; слабоум...
3,фильм рассказывает реальную историю загадочног...
4,пострадав в результате несчастного случая; бог...


In [27]:
#Удаление стоп-слов
#Импорт стоп-слов из бибилиотеки nltk
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('russian'))
# функция, удаляющая стопслова из текстов

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
# Применим функцию удаления стоп-слов
df["text_stop"] = df["text_punct"].apply(stopwords)
df["text_stop"].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text_stop
0,бухгалтер энди дюфрейн обвинён убийстве собств...
1,пол эджкомб — начальник блока смертников тюрьм...
2,лица главного героя форреста гампа; слабоумног...
3,фильм рассказывает реальную историю загадочног...
4,пострадав результате несчастного случая; богат...


In [28]:
#Удаление высокочастотных слов
from collections import Counter
cnt = Counter()
for text in df["text_stop"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('—', 215),
 ('это', 40),
 ('который', 38),
 ('жизнь', 34),
 ('жизни', 30),
 ('своей', 26),
 ('время', 21),
 ('мир', 21),
 ('лет', 21),
 ('вместе', 20)]

In [29]:
#удаление часто встречающиеся слов в корпусе при помощи tf-idf
# Удаление самых частотных слов
freq = set([w for (w, wc) in cnt.most_common(10)])
# функция удаления слов
def freqwords(text):
    return " ".join([word for word in str(text).split() if word not
in freq])
# применение функции
df["text_common"] = df["text_stop"].apply(freqwords)
df["text_common"].head()

,text_common
0,бухгалтер энди дюфрейн обвинён убийстве собств...
1,пол эджкомб начальник блока смертников тюрьме ...
2,лица главного героя форреста гампа; слабоумног...
3,фильм рассказывает реальную историю загадочног...
4,пострадав результате несчастного случая; богат...


In [30]:
#Удаление низкочастотных слов
freq = pd.Series(' '.join(df['text_common']).split()).value_counts()[-10:] # 10 rare words
freq = list(freq.index)
df['text_rare'] = df['text_common'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['text_rare'].head()

,text_rare
0,бухгалтер энди дюфрейн обвинён убийстве собств...
1,пол эджкомб начальник блока смертников тюрьме ...
2,лица главного героя форреста гампа; слабоумног...
3,фильм рассказывает реальную историю загадочног...
4,пострадав результате несчастного случая; богат...


In [31]:
#Удаление эмодзи
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
#passing the emoji function to 'text_rare'
df['text_rare'] = df['text_rare'].apply(emoji)

In [32]:
#Удаление цифр
df['text_nonum'] = df['text_common'].str.replace('\d+', '')
df['text_nonum'].head()

,text_nonum
0,бухгалтер энди дюфрейн обвинён убийстве собств...
1,пол эджкомб начальник блока смертников тюрьме ...
2,лица главного героя форреста гампа; слабоумног...
3,фильм рассказывает реальную историю загадочног...
4,пострадав результате несчастного случая; богат...


In [33]:
#Удаление URL
# Function for url's
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
# Examples
text = "This is my website, https://www.link.com"
remove_urls(text)
#Passing the function to 'text_rare'
df['text_rare'] = df['text_rare'].apply(remove_urls)

In [34]:
#Удаление HTML-тегов
from bs4 import BeautifulSoup
#Function for removing html
def html(text):
    return BeautifulSoup(text, "lxml").text
# Examples
text = """<div>
<h1> Это </h1>
<p> странца</p>
<a href="https://www.link.com/"> пример</a>
</div>
"""
print(html(text))
# Passing the function to 'text_rare'
df['text_rare'] = df['text_rare'].apply(html)


 Это 
 странца
 пример




In [35]:
#Токенизация
def tokenization(text):
    text = re.split('\W+', text)
    return text
# Passing the function to 'text_rare' and store into'text_token'
df['text_token'] = df['text_rare'].apply(lambda x: tokenization(x.lower()))
df[['text_token']].head()

,text_token
0,"[бухгалтер, энди, дюфрейн, обвинён, убийстве, ..."
1,"[пол, эджкомб, начальник, блока, смертников, т..."
2,"[лица, главного, героя, форреста, гампа, слабо..."
3,"[фильм, рассказывает, реальную, историю, загад..."
4,"[пострадав, результате, несчастного, случая, б..."


In [39]:
#Можно выполнить при помощи NLTK. Заодно проведем лемматизацию
!pip install --upgrade pymorphy2
from pymorphy2 import MorphAnalyzer
from nltk import sent_tokenize, word_tokenize, regexp_tokenize

def tokenize_lemmas(sent, pat=r"(?u)\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form
            for tok in regexp_tokenize(sent, pat)]
df["text_lemm"] = df["text_rare"].map(lambda x: " ".join(tokenize_lemmas(x)))
df[['text_lemm']].head()

AttributeError: module 'inspect' has no attribute 'getargspec'

In [40]:
#Исходя из цели автоматической обработки текста, выберем необходимые варианты предобработки текста.
#Например, для классификации можно воспользоваться нижним регистром, удалением  пунктуации, ссылок, стоп-слов,
#произвести лемматизацию и токенизацию
##ИЛИ воспользоваться только удалением ссылок. Т.к. это актуальнее для НС, а выбор предобработки сделал для Машинного обученя
df['text_ready']  = df['overview'].str.lower()
df['text_ready'] = df['text_ready'].str.replace('\d+', '')
df['text_ready'] = df['text_ready'].str.replace('[^\w\s]','')
df["text_ready"] = df["text_ready"].apply(stopwords)
df['text_ready'] = df['text_ready'].apply(emoji)
df['text_ready'] = df['text_ready'].apply(remove_urls)
df['text_ready'] = df['text_ready'].apply(html)
df[['text_ready']].head()

,text_ready
0,бухгалтер энди дюфрейн обвинён убийстве собств...
1,пол эджкомб — начальник блока смертников тюрьм...
2,лица главного героя форреста гампа; слабоумног...
3,фильм рассказывает реальную историю загадочног...
4,пострадав результате несчастного случая; богат...
